# 1. Import libraries

In [ ]:
!pip install -q  evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
# ===== CELL 1: SETUP =====

import os
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import load_from_disk
import evaluate
import gc

# Clear cache
torch.cuda.empty_cache()
gc.collect()

# Set environment variable
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# 2. Load data & model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/NLP_final/NLP_Translation_Project"
QWEN_MODEL = "Qwen/Qwen3-0.6B"

print("✅ Setup complete!")

Mounted at /content/drive
✅ Setup complete!


In [ ]:
# ===== CELL 2: LOAD MODEL & TOKENIZER =====

print(f"📥 Loading {QWEN_MODEL}...")

tokenizer = AutoTokenizer.from_pretrained(
    QWEN_MODEL,
    trust_remote_code=True,
    padding_side='right'
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    QWEN_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False

print(f"✅ Model loaded!")
print(f"   Device: {model.device}")
print(f"   Parameters: {model.num_parameters() / 1e9:.2f}B")


📥 Loading Qwen/Qwen3-0.6B...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✅ Model loaded!
   Device: cuda:0
   Parameters: 0.60B


# 3. Tokenize data

In [ ]:
# ===== CELL 3A: LOAD & TOKENIZE BASELINE DATA =====

# Load baseline dataset
baseline_datasets = load_from_disk(f"{PROJECT_DIR}/dataset_baseline")
print("✅ Baseline dataset loaded!")

MAX_LENGTH = 384  # Optimal for Qwen3 translation

def tokenize_function(examples):
    """
    Tokenize with format: {source}\n{target}{eos}
    """
    texts = []
    for source, target in zip(examples['source'], examples['target']):
        text = f"{source}\n{target}{tokenizer.eos_token}"
        texts.append(text)

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,  # Dynamic padding
        return_tensors=None
    )

    return tokenized

print("🔄 Tokenizing baseline dataset...")
baseline_tokenized = baseline_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=baseline_datasets['train'].column_names,
    desc="Tokenizing baseline"
)

print("\n✅ Baseline tokenization complete!")
print(f"Baseline train: {len(baseline_tokenized['train']):,}")
print(f"Baseline val: {len(baseline_tokenized['validation']):,}")
print(f"Baseline test: {len(baseline_tokenized['test']):,}")


# 4.DATA COLLATOR

In [ ]:

class DataCollatorTranslationTask:
    """
    Custom collator that:
    1. Dynamic padding
    2. Masks source text in labels (only train on target)
    """
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features):
        max_len = max(len(f['input_ids']) for f in features)

        input_ids_list = []
        attention_mask_list = []
        labels_list = []

        for f in features:
            input_ids = f['input_ids']
            attention_mask = f['attention_mask']

            # Pad
            padding_length = max_len - len(input_ids)
            input_ids_padded = input_ids + [self.tokenizer.pad_token_id] * padding_length
            attention_mask_padded = attention_mask + [0] * padding_length

            # Create labels
            labels = input_ids_padded.copy()

            # Find newline position (separator between source and target)
            decoded = self.tokenizer.decode(input_ids)
            newline_pos = decoded.find('\n')

            if newline_pos > 0:
                source_part = decoded[:newline_pos]
                source_tokens = self.tokenizer.encode(source_part, add_special_tokens=False)
                source_len = len(source_tokens)

                # Mask source part (only train on target)
                for i in range(min(source_len + 1, len(labels))):
                    labels[i] = -100

            # Mask padding
            for i in range(len(input_ids), len(labels)):
                labels[i] = -100

            input_ids_list.append(input_ids_padded)
            attention_mask_list.append(attention_mask_padded)
            labels_list.append(labels)

        return {
            'input_ids': torch.tensor(input_ids_list, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask_list, dtype=torch.long),
            'labels': torch.tensor(labels_list, dtype=torch.long),
        }

data_collator = DataCollatorTranslationTask(tokenizer)
print("✅ Data collator ready!")


✅ Data collator ready!


In [ ]:
# ===== CELL 5: COMPUTE METRICS (BLEU) =====

bleu_metric = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    """
    Compute BLEU score during evaluation
    """
    predictions, labels = eval_pred

    # Decode predictions
    if len(predictions.shape) == 3:
        predictions = np.argmax(predictions, axis=-1)

    # Decode
    decoded_preds = []
    decoded_labels = []

    for pred, label in zip(predictions, labels):
        # Remove -100 (padding/masked tokens)
        label = label[label != -100]

        pred_text = tokenizer.decode(pred, skip_special_tokens=True)
        label_text = tokenizer.decode(label, skip_special_tokens=True)

        decoded_preds.append(pred_text)
        decoded_labels.append([label_text])  # BLEU expects list of references

    # Compute BLEU
    result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

    return {
        "bleu": result["score"]
    }

print("✅ Metrics function ready!")


✅ Metrics function ready!


# 5.Training models

## 5.1 Baseline model

In [ ]:
# ===== CELL 7: FULL TRAINING - BASELINE =====
torch.cuda.empty_cache()
gc.collect()

# Reload model để đảm bảo clean
del model
torch.cuda.empty_cache()

model = AutoModelForCausalLM.from_pretrained(
    QWEN_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False
# Enable gradient checkpointing
model.gradient_checkpointing_enable()
print("="*80)
print("🚀 FULL TRAINING - BASELINE MODEL")
print("="*80)

baseline_args = TrainingArguments(
    output_dir=f"{PROJECT_DIR}/models/qwen3_baseline",

    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,


    gradient_checkpointing=True,      # Save memory

    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    bf16=True,
    optim="adamw_torch_fused",
    weight_decay=0.01,
    max_grad_norm=1.0,

    prediction_loss_only=True,

    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    #metric_for_best_model="eval_bleu",
    #greater_is_better=True,

    logging_steps=100,
    dataloader_num_workers=2,
    logging_dir=f"{PROJECT_DIR}/logs/baseline",
    report_to="tensorboard",

    remove_unused_columns=False,
    seed=42,
)
# Custom compute_metrics that clears cache
def compute_metrics_safe(eval_pred):
    torch.cuda.empty_cache()
    return compute_metrics(eval_pred)

baseline_trainer = Trainer(
    model=model,
    args=baseline_args,
    train_dataset=baseline_tokenized['train'],
    eval_dataset=baseline_tokenized['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
)





/tmp/ipython-input-2685849444.py:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  baseline_trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 FULL TRAINING - BASELINE MODEL


In [ ]:

print("\n⏳ Starting training from scratch...")
baseline_result = baseline_trainer.train()

print(f"\n✅ Training complete!")
print(f"   Final train loss: {baseline_result.training_loss:.4f}")

baseline_trainer.save_model(f"{PROJECT_DIR}/models/qwen3_baseline_final")
print(f"\n💾 Saved!")

torch.cuda.empty_cache()
baseline_eval = baseline_trainer.evaluate()
print(f"   Val loss: {baseline_eval['eval_loss']:.4f}")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.



⏳ Starting training from scratch...


Epoch,Training Loss,Validation Loss
1,1.618000,1.550490
2,1.401200,1.540336
3,1.241600,1.551139



✅ Training complete!
   Final train loss: 1.4601

💾 Saved!


   Val loss: 1.5511


## 5.1 Agumented model

In [ ]:
# ===== CELL 3B: LOAD & TOKENIZE AUGMENTED DATA (Run after Notebook 2) =====

# Load augmented dataset
augmented_datasets = load_from_disk(f"{PROJECT_DIR}/dataset_augmented")
print("✅ Augmented dataset loaded!")

MAX_LENGTH = 384  # Optimal for Qwen3 translation

def tokenize_function(examples):
    """
    Tokenize with format: {source}\n{target}{eos}
    """
    texts = []
    for source, target in zip(examples['source'], examples['target']):
        text = f"{source}\n{target}{tokenizer.eos_token}"
        texts.append(text)

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,  # Dynamic padding
        return_tensors=None
    )

    return tokenized

print("🔄 Tokenizing augmented dataset...")
augmented_tokenized = augmented_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=augmented_datasets['train'].column_names,
    desc="Tokenizing augmented"
)

print("\n✅ Augmented tokenization complete!")
print(f"Augmented train: {len(augmented_tokenized['train']):,}")
print(f"Augmented val: {len(augmented_tokenized['validation']):,}")


✅ Augmented dataset loaded!
🔄 Tokenizing augmented dataset...


Tokenizing augmented:   0%|          | 0/2000 [00:00<?, ? examples/s]


✅ Augmented tokenization complete!
Augmented train: 21,426
Augmented val: 2,000


In [ ]:
# ===== CELL 8: RESUME TRAINING - AUGMENTED =====

import torch
import gc
import os

torch.cuda.empty_cache()
gc.collect()

# Check for checkpoint
checkpoint_dir = f"{PROJECT_DIR}/models/qwen3_augmented"
checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith('checkpoint-')]

if checkpoints:
    latest = sorted(checkpoints, key=lambda x: int(x.split('-')[1]))[-1]
    checkpoint_path = os.path.join(checkpoint_dir, latest)
    print(f"📂 Found checkpoint: {latest}")
else:
    checkpoint_path = None
    print("⚠️ No checkpoint found")

# Reload model from checkpoint if exists
if checkpoint_path:
    print(f"🔄 Resuming from {checkpoint_path}...")
    model = AutoModelForCausalLM.from_pretrained(
        checkpoint_path,  # Load from checkpoint
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )
else:
    print("🆕 Starting from pretrained...")
    model = AutoModelForCausalLM.from_pretrained(
        QWEN_MODEL,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

model.config.use_cache = False
model.gradient_checkpointing_enable()

# Same training args
augmented_args = TrainingArguments(
    output_dir=f"{PROJECT_DIR}/models/qwen3_augmented",

    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,

    gradient_checkpointing=True,

    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,

    bf16=True,
    optim="adamw_torch_fused",
    weight_decay=0.01,
    max_grad_norm=1.0,

    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    prediction_loss_only=True,

    logging_steps=100,
    logging_dir=f"{PROJECT_DIR}/logs/augmented",
    report_to="tensorboard",

    remove_unused_columns=False,
    seed=42,
)

augmented_trainer = Trainer(
    model=model,
    args=augmented_args,
    train_dataset=augmented_tokenized['train'],
    eval_dataset=augmented_tokenized['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Resume or start
if checkpoint_path:
    print(f"\n⏳ Resuming from {latest} (Epoch 3)...")
    augmented_result = augmented_trainer.train(resume_from_checkpoint=checkpoint_path)
else:
    print("\n⏳ Starting training...")
    augmented_result = augmented_trainer.train()

print(f"\n✅ Training complete!")
print(f"   Final train loss: {augmented_result.training_loss:.4f}")

# Save
augmented_trainer.save_model(f"{PROJECT_DIR}/models/qwen3_augmented_final")
print(f"\n💾 Saved to qwen3_augmented_final")

# Evaluate
torch.cuda.empty_cache()
augmented_eval = augmented_trainer.evaluate()
print(f"   Val loss: {augmented_eval['eval_loss']:.4f}")


/tmp/ipython-input-3060064903.py:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  augmented_trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 FULL TRAINING - AUGMENTED MODEL (+ Synthetic Data)


In [ ]:
# Train
print("\n⏳ Starting augmented training...")
augmented_result = augmented_trainer.train()

print(f"\n✅ Augmented training complete!")
print(f"   Final train loss: {augmented_result.training_loss:.4f}")

# Save first
augmented_trainer.save_model(f"{PROJECT_DIR}/models/qwen3_augmented_final")
print(f"\n💾 Saved to {PROJECT_DIR}/models/qwen3_augmented_final")

# Evaluate after saving
torch.cuda.empty_cache()
augmented_eval = augmented_trainer.evaluate()
print(f"   Val loss: {augmented_eval['eval_loss']:.4f}")

print("\n💡 Full BLEU will be computed in Notebook 4")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.



⏳ Starting augmented training...


Epoch,Training Loss,Validation Loss
1,1.562400,1.560411
2,1.255000,1.541597


Epoch,Training Loss,Validation Loss
1,1.562400,1.560411
2,1.255000,1.541597
3,1.174500,1.554188


Exception: No such file or directory (os error 2)

In [ ]:
# ===== DISABLE WANDB & EVAL =====

import os
os.environ['WANDB_DISABLED'] = 'true'  # ← DISABLE wandb

import torch
import gc

torch.cuda.empty_cache()
gc.collect()

# Load model from checkpoint-4020
checkpoint_path = f"{PROJECT_DIR}/models/qwen3_augmented/checkpoint-4020"

print(f"📂 Loading from checkpoint-4020...")
model = AutoModelForCausalLM.from_pretrained(
    checkpoint_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False
model.eval()

print("✅ Model loaded!")

# Create trainer WITHOUT wandb
augmented_args = TrainingArguments(
    output_dir=f"{PROJECT_DIR}/models/qwen3_augmented",
    per_device_eval_batch_size=4,
    bf16=True,
    remove_unused_columns=False,
    prediction_loss_only=True,
    report_to="none",  # ← DISABLE ALL LOGGING
)

augmented_trainer = Trainer(
    model=model,
    args=augmented_args,
    eval_dataset=augmented_tokenized['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Evaluate
print("\n📊 Running evaluation...")
torch.cuda.empty_cache()
augmented_eval = augmented_trainer.evaluate()

print(f"\n✅ Evaluation complete!")
print(f"   Val loss: {augmented_eval['eval_loss']:.4f}")

# Save as final model
print("\n💾 Saving as final model...")
augmented_trainer.save_model(f"{PROJECT_DIR}/models/qwen3_augmented_final")
print("✅ Saved to qwen3_augmented_final!")


📂 Loading from checkpoint-4020...


/tmp/ipython-input-1027752526.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  augmented_trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Model loaded!

📊 Running evaluation...



✅ Evaluation complete!
   Val loss: 1.5542

💾 Saving as final model...
✅ Saved to qwen3_augmented_final!
